# Open Weather Map
Exploration of openweathermap.org API.

## Prerequisits
- load libraries
- load configuration file to get API access data
- create URL for API access
- access API
- print general information

In [1]:
import toml
import requests
import pandas as pd

def del_lst(x):
    return x[0]

conf = toml.load('../../conf.toml')
owm = conf['owm']

url = 'https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lng}' \
      '&units=metric&APPID={key}'
url = url.format(lat=owm['lat'], lng=owm['lng'], key=owm['key'])

response = requests.get(url)
dic = response.json()

# keys = ['lat', 'lon', 'timezone', 'timezone_offset']
# gi = pd.DataFrame(list(map(dic.get, keys)), keys, ['General Information']).T
# gi['timezone_offset'] = pd.to_timedelta(gi.timezone_offset, 's')
# gi

## Current Weather
- Timezone is UTC
- temp, feels_like, dew_point: °C
- pressure: hPa
- humidity, clouds: %
- uvi: UV Index
- visibility: m
- wind_speed: m / s
- wind_deg: °

In [2]:
cur = pd.DataFrame(dic['current'])

weather = cur.weather[0]
cur[list(weather.keys())] = weather.values()
cur = cur.drop(columns='weather')

time = ['dt', 'sunrise', 'sunset']
cur[time] = cur[time].apply(pd.to_datetime, unit='s')
cur = cur.set_index('dt')

cur

,sunrise,sunset,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,id,main,description,icon
dt,,,,,,,,,,,,,,,,
2021-06-01 17:36:49,2021-06-01 03:06:48,2021-06-01 19:47:46,18.03,17.06,1017,45,5.93,0.29,75,10000,4.12,40,803,Clouds,broken clouds,04d


### Alerts

In [3]:
# ale = pd.DataFrame(dic['alerts'], ['german', 'english'])
#
# time = ['start', 'end']
# ale[time] = ale[time].apply(pd.to_datetime, unit='s')
#
# ale

## Minutely Data
- 1 hour
- 60 observations
- precipitation: mm

In [4]:
mnt = pd.DataFrame(dic['minutely'])
mnt = mnt.set_index('dt')
mnt.index = pd.to_datetime(mnt.index, unit='s')
mnt

,precipitation
dt,
2021-06-01 17:37:00,0
2021-06-01 17:38:00,0
2021-06-01 17:39:00,0
2021-06-01 17:40:00,0
2021-06-01 17:41:00,0
...,...
2021-06-01 18:33:00,0
2021-06-01 18:34:00,0
2021-06-01 18:35:00,0


## Hourly Data
- 2 days
- 48 observations
- wind_gust: metre/sec
- pop: %
- rain: mm

In [5]:
hr = pd.DataFrame(dic['hourly'])

hr['dt'] = pd.to_datetime(hr.dt, unit='s')
hr = hr.set_index('dt')

hr = hr.join(pd.DataFrame(hr.weather.apply(del_lst).tolist(), hr.index))
hr = hr.drop(columns='weather')
if 'rain' in hr.columns:
    hr.loc[hr.rain.notna(), 'rain'] = hr.loc[hr.rain.notna(), 'rain'].apply(
        lambda x: x.get('1h')
    )
    hr.loc[hr.rain.isna(), 'rain'] = 0
else:
    hr['rain'] = 0

hr

,temp,feels_like,pressure,humidity,dew_point,uvi,clouds,visibility,wind_speed,wind_deg,wind_gust,pop,rain,id,main,description,icon
dt,,,,,,,,,,,,,,,,,
2021-06-01 17:00:00,17.61,16.78,1017,52,7.65,0.74,80,10000,3.52,39,6.05,0.00,0,803,Clouds,broken clouds,04d
2021-06-01 18:00:00,18.03,17.06,1017,45,5.93,0.29,75,10000,3.20,32,5.41,0.00,0,803,Clouds,broken clouds,04d
2021-06-01 19:00:00,16.98,16.07,1017,51,6.78,0.08,80,10000,3.84,33,7.12,0.00,0,803,Clouds,broken clouds,04d
2021-06-01 20:00:00,15.65,14.71,1018,55,6.65,0.00,85,10000,3.14,42,4.84,0.00,0,804,Clouds,overcast clouds,04n
2021-06-01 21:00:00,14.28,13.43,1018,64,7.57,0.00,90,10000,3.24,50,3.99,0.00,0,804,Clouds,overcast clouds,04n
2021-06-01 22:00:00,12.81,12.26,1019,81,9.63,0.00,95,10000,3.28,60,5.08,0.00,0,804,Clouds,overcast clouds,04n
2021-06-01 23:00:00,11.30,10.84,1019,90,9.74,0.00,100,10000,3.06,76,3.77,0.00,0,804,Clouds,overcast clouds,04n
2021-06-02 00:00:00,10.49,9.84,1018,86,8.35,0.00,97,10000,3.44,74,4.38,0.00,0,804,Clouds,overcast clouds,04n
2021-06-02 01:00:00,9.82,8.11,1018,86,7.67,0.00,88,10000,3.33,81,4.43,0.00,0,804,Clouds,overcast clouds,04n


## Daily Data
- 1 week
- 8 observations

In [6]:
time = ['dt', 'sunrise', 'sunset', 'moonrise', 'moonset']
dly = pd.DataFrame(dic['daily'])
dly[time] = dly[time].apply(pd.to_datetime, unit='s')

dly = dly.join(pd.DataFrame(dly.temp.to_list()))
dly = dly.join(pd.DataFrame(dly.feels_like.to_list()).add_suffix('_feels_like'))
dly = dly.join(pd.DataFrame(dly.weather.apply(del_lst).to_list()))

dly = dly.drop(columns=['temp', 'feels_like', 'weather'])
dly = dly.set_index('dt')
dly

,sunrise,sunset,moonrise,moonset,moon_phase,pressure,humidity,dew_point,wind_speed,wind_deg,...,eve,morn,day_feels_like,night_feels_like,eve_feels_like,morn_feels_like,id,main,description,icon
dt,,,,,,,,,,,,,,,,,,,,,
2021-06-01 11:00:00,2021-06-01 03:06:48,2021-06-01 19:47:46,2021-06-01 00:32:00,2021-06-01 09:32:00,0.72,1020,66,12.88,4.41,49,...,17.61,12.58,19.21,13.43,16.78,12.19,803,Clouds,broken clouds,04d
2021-06-02 11:00:00,2021-06-02 03:05:56,2021-06-02 19:48:57,2021-06-02 00:50:00,2021-06-02 10:50:00,0.75,1019,65,11.09,6.02,96,...,18.83,10.52,17.26,13.62,18.75,10.00,501,Rain,moderate rain,10d
2021-06-03 11:00:00,2021-06-03 03:05:07,2021-06-03 19:50:05,2021-06-03 01:05:00,2021-06-03 12:05:00,0.79,1021,74,16.69,4.35,137,...,21.07,14.21,21.68,16.42,21.32,14.09,501,Rain,moderate rain,10d
2021-06-04 11:00:00,2021-06-04 03:04:21,2021-06-04 19:51:11,2021-06-04 01:17:00,2021-06-04 13:17:00,0.82,1022,83,16.69,3.39,73,...,19.16,14.66,19.94,15.16,19.43,14.71,500,Rain,light rain,10d
2021-06-05 11:00:00,2021-06-05 03:03:38,2021-06-05 19:52:14,2021-06-05 01:29:00,2021-06-05 14:27:00,0.85,1022,77,15.88,4.00,70,...,18.36,15.20,20.14,15.09,18.58,15.10,500,Rain,light rain,10d
2021-06-06 11:00:00,2021-06-06 03:02:58,2021-06-06 19:53:14,2021-06-06 01:40:00,2021-06-06 15:39:00,0.88,1021,72,15.30,4.04,74,...,18.68,15.12,20.72,15.14,18.82,15.04,500,Rain,light rain,10d
2021-06-07 11:00:00,2021-06-07 03:02:22,2021-06-07 19:54:12,2021-06-07 01:53:00,2021-06-07 16:49:00,0.91,1022,66,14.89,3.36,111,...,19.25,16.04,21.47,15.26,19.35,15.89,500,Rain,light rain,10d
2021-06-08 11:00:00,2021-06-08 03:01:49,2021-06-08 19:55:07,2021-06-08 02:08:00,2021-06-08 18:01:00,0.94,1020,63,14.64,3.88,20,...,18.64,16.59,22.10,15.40,18.75,16.47,501,Rain,moderate rain,10d
